# 🚀 YOLOv8 Professional Training Pipeline
## CSV → YOLO Conversion · Training · Evaluation · Export · Repository Builder

**Features:**
- Auto-detect CSV annotation format (pixel boxes, width/height, normalized YOLO)
- Deterministic train/val split with configurable ratio
- Full YOLOv8 training with Ultralytics
- Evaluation, inference visualization, and model export (ONNX, TorchScript)
- **Generates a complete professional repository** for GitHub & Hugging Face

---
> 📌 **How to use:** Run cells sequentially. Edit the **Configuration** cell to match your dataset.

## 1 · Install Dependencies

In [ ]:
# ── Install pinned dependencies ──────────────────────────────────
!pip install -q ultralytics==8.2.103 opencv-python-headless==4.10.0.84 \
    pandas==2.2.2 pyyaml==6.0.2 matplotlib==3.9.2 onnx==1.16.2 \
    onnxruntime==1.19.2 Pillow==10.4.0 tqdm==4.66.5 seaborn==0.13.2

import ultralytics
print(f"Ultralytics version: {ultralytics.__version__}")

## 2 · Configuration

Edit the paths and hyperparameters below to match your dataset.

In [ ]:
# ══════════════════════════════════════════════════════════════════
# CONFIGURATION — edit these values
# ══════════════════════════════════════════════════════════════════

# ── Paths ────────────────────────────────────────────────────────
IMAGES_DIR   = "/content/data/images"       # folder with jpg/png images
CSV_PATH     = "/content/data/annotations.csv"  # CSV annotation file
WORK_DIR     = "/content/work"              # intermediate files
OUTPUT_DIR   = "/content/output"            # final artifacts
REPO_DIR     = "/content/repo"              # generated repository

# ── Train / Val split ────────────────────────────────────────────
SPLIT_RATIO  = 0.8     # fraction for training (rest → validation)
SEED         = 42

# ── Training hyperparameters ─────────────────────────────────────
MODEL_SIZE   = "yolov8n.pt"   # yolov8n / yolov8s / yolov8m / yolov8l / yolov8x
IMGSZ        = 640
EPOCHS       = 50
BATCH        = 16
DEVICE       = ""             # "" = auto, "0" = GPU 0, "cpu" = CPU

# ── Options ──────────────────────────────────────────────────────
ALLOW_BACKGROUND_IMAGES = True   # keep images with no labels as background
LICENSE_TYPE = "MIT"              # "MIT" or "Apache-2.0"

# ══════════════════════════════════════════════════════════════════
print("✅ Configuration loaded.")

## 3 · Imports & Utility Functions

In [ ]:
import os, shutil, random, glob, pathlib, warnings, sys, yaml, csv, json
from pathlib import Path
from collections import Counter, OrderedDict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")
random.seed(SEED)
np.random.seed(SEED)

# ── Helper: get image dimensions ─────────────────────────────────
def get_image_size(path):
    """Return (width, height) of an image file."""
    with Image.open(path) as img:
        return img.size  # (w, h)

# ── Helper: detect CSV format ────────────────────────────────────
def detect_csv_format(df):
    """
    Auto-detect CSV annotation format.
    Returns one of: 'A' (xmin,ymin,xmax,ymax), 'B' (xmin,ymin,w,h),
                     'C' (normalized YOLO), or raises ValueError.
    """
    cols_lower = [c.lower().strip() for c in df.columns]

    # Check for 'normalized' column → Format C
    if 'normalized' in cols_lower:
        print("📐 Detected Format C (normalized YOLO-like)")
        return 'C'

    # Check column names for xmax/ymax → Format A
    if 'xmax' in cols_lower and 'ymax' in cols_lower:
        print("📐 Detected Format A (xmin, ymin, xmax, ymax)")
        return 'A'

    # Check for width/height columns → Format B
    if 'width' in cols_lower and 'height' in cols_lower:
        print("📐 Detected Format B (xmin, ymin, width, height)")
        return 'B'

    # Fallback: try positional detection
    if len(df.columns) >= 6:
        # Check if values look normalized (all between 0 and 1)
        numeric_cols = df.iloc[:, 1:5]
        try:
            numeric_vals = numeric_cols.astype(float)
            if numeric_vals.max().max() <= 1.0 and numeric_vals.min().min() >= 0.0:
                print("📐 Auto-detected Format C (values appear normalized)")
                return 'C'
        except (ValueError, TypeError):
            pass

        print("📐 Falling back to Format A (positional: col1-4 as xmin,ymin,xmax,ymax)")
        return 'A'

    raise ValueError(
        "❌ Cannot detect CSV format. Expected columns like:\n"
        "  Format A: filename, xmin, ymin, xmax, ymax, class\n"
        "  Format B: filename, xmin, ymin, width, height, class\n"
        "  Format C: filename, x_center, y_center, width, height, class, normalized"
    )

# ── Helper: normalize columns ────────────────────────────────────
def normalize_columns(df, fmt):
    """Rename columns to a standard set based on detected format."""
    df = df.copy()
    cols = list(df.columns)

    if fmt == 'A':
        mapping = {}
        cols_lower = {c.lower().strip(): c for c in cols}
        for target, candidates in [
            ('filename', ['filename', 'file', 'image', 'image_name', 'img']),
            ('xmin', ['xmin', 'x_min', 'x1', 'left']),
            ('ymin', ['ymin', 'y_min', 'y1', 'top']),
            ('xmax', ['xmax', 'x_max', 'x2', 'right']),
            ('ymax', ['ymax', 'y_max', 'y2', 'bottom']),
            ('class', ['class', 'label', 'class_name', 'category', 'cls']),
        ]:
            for c in candidates:
                if c in cols_lower:
                    mapping[cols_lower[c]] = target
                    break
        if len(mapping) < 6:
            # positional fallback
            mapping = {cols[0]: 'filename', cols[1]: 'xmin', cols[2]: 'ymin',
                       cols[3]: 'xmax', cols[4]: 'ymax', cols[5]: 'class'}
        df = df.rename(columns=mapping)

    elif fmt == 'B':
        cols_lower = {c.lower().strip(): c for c in cols}
        mapping = {}
        for target, candidates in [
            ('filename', ['filename', 'file', 'image', 'image_name', 'img']),
            ('xmin', ['xmin', 'x_min', 'x1', 'left', 'x']),
            ('ymin', ['ymin', 'y_min', 'y1', 'top', 'y']),
            ('width', ['width', 'w', 'box_width']),
            ('height', ['height', 'h', 'box_height']),
            ('class', ['class', 'label', 'class_name', 'category', 'cls']),
        ]:
            for c in candidates:
                if c in cols_lower:
                    mapping[cols_lower[c]] = target
                    break
        if len(mapping) < 6:
            mapping = {cols[0]: 'filename', cols[1]: 'xmin', cols[2]: 'ymin',
                       cols[3]: 'width', cols[4]: 'height', cols[5]: 'class'}
        df = df.rename(columns=mapping)

    elif fmt == 'C':
        cols_lower = {c.lower().strip(): c for c in cols}
        mapping = {}
        for target, candidates in [
            ('filename', ['filename', 'file', 'image', 'image_name', 'img']),
            ('x_center', ['x_center', 'cx', 'center_x', 'x']),
            ('y_center', ['y_center', 'cy', 'center_y', 'y']),
            ('width', ['width', 'w', 'box_width']),
            ('height', ['height', 'h', 'box_height']),
            ('class', ['class', 'label', 'class_name', 'category', 'cls']),
        ]:
            for c in candidates:
                if c in cols_lower:
                    mapping[cols_lower[c]] = target
                    break
        if len(mapping) < 6:
            mapping = {cols[0]: 'filename', cols[1]: 'x_center', cols[2]: 'y_center',
                       cols[3]: 'width', cols[4]: 'height', cols[5]: 'class'}
        df = df.rename(columns=mapping)

    # Normalize filename to basename
    df['filename'] = df['filename'].apply(lambda x: os.path.basename(str(x).strip()))
    return df

# ── Helper: convert to YOLO format ───────────────────────────────
def convert_to_yolo(row, fmt, img_w, img_h):
    """
    Convert a single annotation row to YOLO format:
    (x_center, y_center, width, height) all normalized to [0, 1].
    Returns None if the box is invalid.
    """
    try:
        if fmt == 'A':
            xmin = float(row['xmin'])
            ymin = float(row['ymin'])
            xmax = float(row['xmax'])
            ymax = float(row['ymax'])
        elif fmt == 'B':
            xmin = float(row['xmin'])
            ymin = float(row['ymin'])
            xmax = xmin + float(row['width'])
            ymax = ymin + float(row['height'])
        elif fmt == 'C':
            # Already normalized
            xc = float(row['x_center'])
            yc = float(row['y_center'])
            w  = float(row['width'])
            h  = float(row['height'])
            # Clamp
            xc = max(0.0, min(1.0, xc))
            yc = max(0.0, min(1.0, yc))
            w  = max(0.0, min(1.0, w))
            h  = max(0.0, min(1.0, h))
            if w <= 0 or h <= 0:
                return None
            return (xc, yc, w, h)

        # For A and B: clamp to image bounds
        xmin = max(0, min(img_w, xmin))
        ymin = max(0, min(img_h, ymin))
        xmax = max(0, min(img_w, xmax))
        ymax = max(0, min(img_h, ymax))

        bw = xmax - xmin
        bh = ymax - ymin
        if bw <= 0 or bh <= 0:
            return None

        xc = (xmin + xmax) / 2.0 / img_w
        yc = (ymin + ymax) / 2.0 / img_h
        w  = bw / img_w
        h  = bh / img_h
        return (xc, yc, w, h)

    except (ValueError, TypeError):
        return None

# ── Helper: build class mapping ──────────────────────────────────
def build_class_mapping(classes_series):
    """
    Build a mapping from class label to integer ID.
    If labels are already integers, validate contiguous from 0; remap if needed.
    Returns: dict {label: int_id}, list of class names ordered by id.
    """
    unique_classes = classes_series.unique()

    # Check if all are integers
    all_int = True
    for c in unique_classes:
        try:
            int(c)
        except (ValueError, TypeError):
            all_int = False
            break

    if all_int:
        int_classes = sorted([int(c) for c in unique_classes])
        if int_classes == list(range(len(int_classes))):
            mapping = {str(c): c for c in int_classes}
            names = [str(c) for c in int_classes]
        else:
            # Remap
            mapping = {str(c): i for i, c in enumerate(int_classes)}
            names = [str(c) for c in int_classes]
            print(f"⚠️  Class IDs are not contiguous. Remapped: {mapping}")
    else:
        sorted_classes = sorted([str(c) for c in unique_classes])
        mapping = {c: i for i, c in enumerate(sorted_classes)}
        names = sorted_classes

    return mapping, names

# ── Helper: visualize boxes ──────────────────────────────────────
def visualize_yolo_labels(image_path, label_path, class_names, ax=None):
    """Draw YOLO bounding boxes on an image."""
    img = Image.open(image_path)
    w, h = img.size
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.imshow(img)
    ax.set_title(os.path.basename(image_path), fontsize=9)
    ax.axis('off')

    colors = plt.cm.Set3(np.linspace(0, 1, max(len(class_names), 1)))

    if os.path.exists(label_path):
        with open(label_path) as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                cls_id = int(parts[0])
                xc, yc, bw, bh = [float(x) for x in parts[1:]]
                x1 = (xc - bw / 2) * w
                y1 = (yc - bh / 2) * h
                rect_w = bw * w
                rect_h = bh * h
                color = colors[cls_id % len(colors)]
                rect = patches.Rectangle((x1, y1), rect_w, rect_h,
                                         linewidth=2, edgecolor=color,
                                         facecolor='none')
                ax.add_patch(rect)
                label = class_names[cls_id] if cls_id < len(class_names) else str(cls_id)
                ax.text(x1, y1 - 4, label, fontsize=8, color='white',
                        bbox=dict(boxstyle='round,pad=0.2', facecolor=color, alpha=0.8))
    return ax

print("✅ Utility functions loaded.")

## 4 · Data Validation & Inspection

Read the CSV, auto-detect format, validate data, and display summary statistics.

In [ ]:
# ── Read CSV ─────────────────────────────────────────────────────
df = pd.read_csv(CSV_PATH)
print(f"📄 CSV loaded: {len(df)} rows, {len(df.columns)} columns")
print(f"   Columns: {list(df.columns)}")
print()
df.head(10)

In [ ]:
# ── Detect format & normalize columns ────────────────────────────
csv_format = detect_csv_format(df)
df = normalize_columns(df, csv_format)
print(f"\n📊 Format: {csv_format}")
print(f"   Unique classes: {df['class'].nunique()}")
print(f"   Class distribution:")
print(df['class'].value_counts().to_string())
print(f"\n   Unique images in CSV: {df['filename'].nunique()}")

In [ ]:
# ── Validate image files exist ───────────────────────────────────
image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff', '.webp'}
available_images = {}
for f in os.listdir(IMAGES_DIR):
    if pathlib.Path(f).suffix.lower() in image_extensions:
        available_images[f] = os.path.join(IMAGES_DIR, f)

csv_filenames = set(df['filename'].unique())
found = csv_filenames & set(available_images.keys())
missing = csv_filenames - set(available_images.keys())

print(f"🖼️  Images in folder: {len(available_images)}")
print(f"   Matched to CSV:   {len(found)}")
if missing:
    print(f"   ⚠️  Missing files:  {len(missing)}")
    for m in list(missing)[:10]:
        print(f"      - {m}")
    if len(missing) > 10:
        print(f"      ... and {len(missing) - 10} more")
    # Remove rows with missing images
    df = df[df['filename'].isin(found)]
    print(f"   → Kept {len(df)} rows after removing missing images.")

assert len(found) > 0, "❌ No matching images found. Check IMAGES_DIR and CSV filenames."
print("\n✅ Data validation passed.")

In [ ]:
# ── Validate & clean bounding boxes ──────────────────────────────
initial_count = len(df)
clamped_count = 0

if csv_format in ('A', 'B'):
    # Check for NaN in coordinate columns
    coord_cols = ['xmin', 'ymin']
    coord_cols += ['xmax', 'ymax'] if csv_format == 'A' else ['width', 'height']
    df = df.dropna(subset=coord_cols + ['class', 'filename'])

    # Convert to numeric
    for col in coord_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df = df.dropna(subset=coord_cols)

elif csv_format == 'C':
    coord_cols = ['x_center', 'y_center', 'width', 'height']
    df = df.dropna(subset=coord_cols + ['class', 'filename'])
    for col in coord_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df = df.dropna(subset=coord_cols)

dropped = initial_count - len(df)
if dropped > 0:
    print(f"⚠️  Dropped {dropped} rows with NaN/invalid values.")

print(f"✅ {len(df)} valid annotation rows remaining.")
print(f"   Covering {df['filename'].nunique()} images.")

## 5 · YOLO Dataset Conversion Pipeline

Create the YOLO directory structure, split into train/val, convert annotations, and generate `data.yaml`.

In [ ]:
# ── Build class mapping ──────────────────────────────────────────
class_mapping, class_names = build_class_mapping(df['class'])
num_classes = len(class_names)
print(f"🏷️  {num_classes} classes: {class_names}")
print(f"   Mapping: {class_mapping}")

In [ ]:
# ── Create YOLO directory structure ──────────────────────────────
DATASET_DIR = os.path.join(WORK_DIR, "dataset")
for split in ['train', 'val']:
    os.makedirs(os.path.join(DATASET_DIR, 'images', split), exist_ok=True)
    os.makedirs(os.path.join(DATASET_DIR, 'labels', split), exist_ok=True)

# ── Split images into train / val ────────────────────────────────
all_images = sorted(df['filename'].unique())
random.seed(SEED)
random.shuffle(all_images)
split_idx = int(len(all_images) * SPLIT_RATIO)
train_images = set(all_images[:split_idx])
val_images   = set(all_images[split_idx:])

# Handle background images (images in folder but not in CSV)
if ALLOW_BACKGROUND_IMAGES:
    bg_images = set(available_images.keys()) - csv_filenames
    if bg_images:
        bg_list = sorted(bg_images)
        random.shuffle(bg_list)
        bg_split = int(len(bg_list) * SPLIT_RATIO)
        train_images |= set(bg_list[:bg_split])
        val_images   |= set(bg_list[bg_split:])
        print(f"🖼️  Added {len(bg_images)} background images (no labels).")

print(f"📂 Train: {len(train_images)} images | Val: {len(val_images)} images")

In [ ]:
# ── Convert annotations & copy images ────────────────────────────
stats = {'train': 0, 'val': 0, 'boxes': 0, 'skipped_boxes': 0}

for img_name in tqdm(sorted(train_images | val_images), desc="Converting"):
    split = 'train' if img_name in train_images else 'val'
    stats[split] += 1

    # Copy image
    src_path = available_images.get(img_name)
    if src_path is None:
        continue
    dst_img = os.path.join(DATASET_DIR, 'images', split, img_name)
    if not os.path.exists(dst_img):
        shutil.copy2(src_path, dst_img)

    # Get image dimensions
    img_w, img_h = get_image_size(src_path)

    # Get annotations for this image
    img_df = df[df['filename'] == img_name]

    # Write label file
    label_name = os.path.splitext(img_name)[0] + '.txt'
    label_path = os.path.join(DATASET_DIR, 'labels', split, label_name)

    lines = []
    for _, row in img_df.iterrows():
        result = convert_to_yolo(row, csv_format, img_w, img_h)
        if result is None:
            stats['skipped_boxes'] += 1
            continue
        xc, yc, w, h = result
        cls_id = class_mapping[str(row['class'])]
        lines.append(f"{cls_id} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}")
        stats['boxes'] += 1

    with open(label_path, 'w') as f:
        f.write("\n".join(lines))

print(f"\n✅ Conversion complete!")
print(f"   Train images: {stats['train']}")
print(f"   Val images:   {stats['val']}")
print(f"   Total boxes:  {stats['boxes']}")
if stats['skipped_boxes'] > 0:
    print(f"   ⚠️  Skipped boxes: {stats['skipped_boxes']}")

In [ ]:
# ── Generate data.yaml ───────────────────────────────────────────
data_yaml = {
    'path': DATASET_DIR,
    'train': 'images/train',
    'val': 'images/val',
    'nc': num_classes,
    'names': class_names,
}

data_yaml_path = os.path.join(DATASET_DIR, 'data.yaml')
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False, sort_keys=False)

print(f"📄 data.yaml written to: {data_yaml_path}")
print("─" * 50)
with open(data_yaml_path) as f:
    print(f.read())

## 6 · Visualization

Display sample training images with bounding boxes to verify the conversion.

In [ ]:
# ── Visualize sample training images ─────────────────────────────
train_img_dir = os.path.join(DATASET_DIR, 'images', 'train')
train_lbl_dir = os.path.join(DATASET_DIR, 'labels', 'train')

sample_images = [f for f in os.listdir(train_img_dir)
                 if pathlib.Path(f).suffix.lower() in image_extensions]

# Prefer images that have labels
labeled = [f for f in sample_images
           if os.path.getsize(os.path.join(train_lbl_dir,
              os.path.splitext(f)[0] + '.txt')) > 0
           if os.path.exists(os.path.join(train_lbl_dir,
              os.path.splitext(f)[0] + '.txt'))]

display_images = labeled[:5] if len(labeled) >= 5 else sample_images[:5]
n = len(display_images)

fig, axes = plt.subplots(1, n, figsize=(5 * n, 5))
if n == 1:
    axes = [axes]

for ax, img_name in zip(axes, display_images):
    img_path = os.path.join(train_img_dir, img_name)
    lbl_path = os.path.join(train_lbl_dir, os.path.splitext(img_name)[0] + '.txt')
    visualize_yolo_labels(img_path, lbl_path, class_names, ax=ax)

plt.suptitle("Sample Training Images with YOLO Bounding Boxes", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 7 · YOLOv8 Training

Train the model using Ultralytics. Metrics are logged automatically.

In [ ]:
from ultralytics import YOLO

# ── Initialize model ─────────────────────────────────────────────
model = YOLO(MODEL_SIZE)
print(f"🔧 Model: {MODEL_SIZE}")
print(f"   Image size: {IMGSZ}")
print(f"   Epochs: {EPOCHS}")
print(f"   Batch: {BATCH}")
print(f"   Device: {DEVICE if DEVICE else 'auto'}")

# ── Train ────────────────────────────────────────────────────────
results = model.train(
    data=data_yaml_path,
    imgsz=IMGSZ,
    epochs=EPOCHS,
    batch=BATCH,
    device=DEVICE if DEVICE else None,
    seed=SEED,
    project=os.path.join(WORK_DIR, "runs"),
    name="train",
    exist_ok=True,
    verbose=True,
)

print("\n✅ Training complete!")

In [ ]:
# ── Display training metrics ─────────────────────────────────────
train_dir = os.path.join(WORK_DIR, "runs", "train")

# Show results plots if available
for plot_name in ['results.png', 'confusion_matrix.png', 'confusion_matrix_normalized.png']:
    plot_path = os.path.join(train_dir, plot_name)
    if os.path.exists(plot_path):
        img = Image.open(plot_path)
        fig, ax = plt.subplots(figsize=(12, 8) if 'results' in plot_name else (8, 8))
        ax.imshow(img)
        ax.axis('off')
        ax.set_title(plot_name, fontsize=14)
        plt.tight_layout()
        plt.show()

## 8 · Evaluation & Inference

Run validation and display predictions on sample images.

In [ ]:
# ── Validation ───────────────────────────────────────────────────
best_weights = os.path.join(train_dir, "weights", "best.pt")
model_best = YOLO(best_weights)

val_results = model_best.val(data=data_yaml_path, imgsz=IMGSZ, verbose=True)

print("\n📊 Validation Results:")
print(f"   mAP50:    {val_results.box.map50:.4f}")
print(f"   mAP50-95: {val_results.box.map:.4f}")
print(f"   Precision: {val_results.box.mp:.4f}")
print(f"   Recall:    {val_results.box.mr:.4f}")

In [ ]:
# ── Inference on sample validation images ────────────────────────
val_img_dir = os.path.join(DATASET_DIR, 'images', 'val')
val_samples = [f for f in os.listdir(val_img_dir)
               if pathlib.Path(f).suffix.lower() in image_extensions][:4]

if val_samples:
    fig, axes = plt.subplots(1, len(val_samples), figsize=(5 * len(val_samples), 5))
    if len(val_samples) == 1:
        axes = [axes]

    for ax, img_name in zip(axes, val_samples):
        img_path = os.path.join(val_img_dir, img_name)
        preds = model_best.predict(img_path, imgsz=IMGSZ, conf=0.25, verbose=False)
        # Plot using ultralytics built-in
        result_img = preds[0].plot()
        ax.imshow(result_img[..., ::-1])  # BGR to RGB
        ax.set_title(img_name, fontsize=9)
        ax.axis('off')

    plt.suptitle("Predictions on Validation Images", fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
else:
    print("⚠️  No validation images found for inference preview.")

## 9 · Model Export

Export the best model to ONNX and TorchScript formats, and save all artifacts.

In [ ]:
# ── Export to ONNX ───────────────────────────────────────────────
os.makedirs(OUTPUT_DIR, exist_ok=True)

try:
    onnx_path = model_best.export(format="onnx", imgsz=IMGSZ)
    print(f"✅ ONNX exported: {onnx_path}")
except Exception as e:
    print(f"⚠️  ONNX export failed: {e}")
    onnx_path = None

# ── Export to TorchScript ────────────────────────────────────────
try:
    ts_path = model_best.export(format="torchscript", imgsz=IMGSZ)
    print(f"✅ TorchScript exported: {ts_path}")
except Exception as e:
    print(f"⚠️  TorchScript export failed: {e}")
    ts_path = None

# ── Copy artifacts to output directory ───────────────────────────
shutil.copy2(best_weights, os.path.join(OUTPUT_DIR, "best.pt"))
shutil.copy2(data_yaml_path, os.path.join(OUTPUT_DIR, "data.yaml"))

if onnx_path and os.path.exists(str(onnx_path)):
    shutil.copy2(str(onnx_path), os.path.join(OUTPUT_DIR, "best.onnx"))
if ts_path and os.path.exists(str(ts_path)):
    shutil.copy2(str(ts_path), os.path.join(OUTPUT_DIR, "best.torchscript"))

# Save class names
with open(os.path.join(OUTPUT_DIR, "classes.json"), 'w') as f:
    json.dump({"names": class_names, "mapping": class_mapping}, f, indent=2)

# Save training config
train_config = {
    "model": MODEL_SIZE, "imgsz": IMGSZ, "epochs": EPOCHS,
    "batch": BATCH, "seed": SEED, "split_ratio": SPLIT_RATIO,
    "num_classes": num_classes, "class_names": class_names,
}
with open(os.path.join(OUTPUT_DIR, "train_config.json"), 'w') as f:
    json.dump(train_config, f, indent=2)

print(f"\n📦 Artifacts saved to: {OUTPUT_DIR}")
for f in sorted(os.listdir(OUTPUT_DIR)):
    size = os.path.getsize(os.path.join(OUTPUT_DIR, f))
    print(f"   {f:30s} {size / 1024:.1f} KB")

## 10 · Professional Repository Generation

Generate a complete, ready-to-use repository structure for **GitHub** and **Hugging Face**.

In [ ]:
# ══════════════════════════════════════════════════════════════════
# REPOSITORY STRUCTURE GENERATION
# ══════════════════════════════════════════════════════════════════

import textwrap as _tw

# ── Create directory structure ────────────────────────────────────
repo_dirs = [
    REPO_DIR,
    f"{REPO_DIR}/src/yolov8_pipeline",
    f"{REPO_DIR}/scripts",
    f"{REPO_DIR}/configs",
    f"{REPO_DIR}/docs",
    f"{REPO_DIR}/artifacts",
    f"{REPO_DIR}/assets",
    f"{REPO_DIR}/huggingface",
]
for d in repo_dirs:
    os.makedirs(d, exist_ok=True)

print("📁 Repository directory structure created:")
for d in repo_dirs:
    rel = os.path.relpath(d, REPO_DIR)
    depth = rel.count(os.sep)
    print(f"   {'  ' * depth}📂 {os.path.basename(d)}/")

In [ ]:
# ── README.md ────────────────────────────────────────────────────
readme_content = f"""# 🚀 YOLOv8 Object Detection Pipeline

A professional, end-to-end pipeline for training **YOLOv8** object detection models using CSV-formatted annotations. Supports multiple annotation formats with automatic detection, deterministic train/val splitting, and comprehensive model evaluation.

![Python](https://img.shields.io/badge/Python-3.8%2B-blue)
![Ultralytics](https://img.shields.io/badge/Ultralytics-8.2-purple)
![License](https://img.shields.io/badge/License-{LICENSE_TYPE}-green)

---

## 📋 Table of Contents

- [Features](#-features)
- [Quick Start](#-quick-start)
- [Data Format](#-data-format)
- [Project Structure](#-project-structure)
- [Training](#-training)
- [Evaluation](#-evaluation)
- [Inference](#-inference)
- [Export](#-export)
- [Configuration](#-configuration)
- [Troubleshooting](#-troubleshooting)
- [License](#-license)

---

## ✨ Features

- **Auto-detect CSV format**: Supports pixel boxes (xmin/ymin/xmax/ymax), width/height format, and normalized YOLO format
- **Robust data validation**: Checks for missing images, invalid boxes, NaN values, and out-of-bound coordinates
- **Deterministic splitting**: Reproducible train/val splits with configurable seed
- **Background image support**: Optionally include unlabeled images as background samples
- **Multi-format export**: ONNX, TorchScript, and PyTorch weights
- **Professional structure**: Ready for GitHub and Hugging Face deployment

---

## 🚀 Quick Start

### Option 1: Google Colab (Recommended)

Open the included notebook and run all cells:

```
YOLOv8_Professional_Pipeline.ipynb
```

### Option 2: Command Line

```bash
# 1. Install dependencies
pip install -r requirements.txt

# 2. Convert CSV annotations to YOLO format
python scripts/convert_csv_to_yolo.py --config configs/default.yaml

# 3. Train the model
python scripts/train.py --config configs/default.yaml

# 4. Run predictions
python scripts/predict.py --weights artifacts/best.pt --source path/to/images
```

---

## 📊 Data Format

Place your data in the following structure:

```
data/
├── images/          # Your image files (jpg, png, etc.)
│   ├── img001.jpg
│   ├── img002.jpg
│   └── ...
└── annotations.csv  # Bounding box annotations
```

### Supported CSV Formats

| Format | Columns | Description |
|--------|---------|-------------|
| **A** | `filename, xmin, ymin, xmax, ymax, class` | Pixel coordinates (corners) |
| **B** | `filename, xmin, ymin, width, height, class` | Pixel coordinates (top-left + size) |
| **C** | `filename, x_center, y_center, width, height, class, normalized` | Normalized YOLO format |

> The format is **auto-detected** from column names. See `docs/dataset_format.md` for details.

---

## 📁 Project Structure

```
├── README.md                    # This file
├── requirements.txt             # Python dependencies
├── pyproject.toml               # Project metadata
├── Makefile                     # Common commands
├── .gitignore                   # Git ignore rules
├── LICENSE                      # License file
├── configs/
│   └── default.yaml             # Default configuration
├── src/yolov8_pipeline/
│   ├── __init__.py
│   ├── parser.py                # CSV parsing & format detection
│   ├── converter.py             # YOLO format conversion
│   └── visualizer.py            # Visualization utilities
├── scripts/
│   ├── convert_csv_to_yolo.py   # CSV → YOLO conversion script
│   ├── train.py                 # Training script
│   └── predict.py               # Inference script
├── docs/
│   ├── dataset_format.md        # Dataset format guide
│   └── usage_guide.md           # Detailed usage guide
├── artifacts/                   # Trained model weights & configs
│   ├── best.pt
│   ├── data.yaml
│   └── classes.json
├── assets/                      # Sample images
└── huggingface/
    └── README.md                # Hugging Face model card
```

---

## 🏋️ Training

### Using the Config File

```bash
python scripts/train.py --config configs/default.yaml
```

### Using Command Line Arguments

```bash
python scripts/train.py \\
    --data path/to/data.yaml \\
    --model yolov8n.pt \\
    --imgsz 640 \\
    --epochs 50 \\
    --batch 16
```

### Training Parameters

| Parameter | Default | Description |
|-----------|---------|-------------|
| `model` | `yolov8n.pt` | Model size (n/s/m/l/x) |
| `imgsz` | `640` | Input image size |
| `epochs` | `50` | Number of training epochs |
| `batch` | `16` | Batch size |
| `seed` | `42` | Random seed |
| `device` | `auto` | Device (cuda/cpu) |

---

## 📈 Evaluation

```bash
python -c "from ultralytics import YOLO; m = YOLO('artifacts/best.pt'); m.val(data='artifacts/data.yaml')"
```

Key metrics reported:
- **mAP50**: Mean Average Precision at IoU 0.50
- **mAP50-95**: Mean Average Precision at IoU 0.50-0.95
- **Precision**: Detection precision
- **Recall**: Detection recall

---

## 🔍 Inference

```bash
python scripts/predict.py \\
    --weights artifacts/best.pt \\
    --source path/to/images \\
    --conf 0.25 \\
    --imgsz 640
```

### Python API

```python
from ultralytics import YOLO

model = YOLO("artifacts/best.pt")
results = model.predict("image.jpg", conf=0.25)

for r in results:
    boxes = r.boxes
    for box in boxes:
        print(f"Class: {{box.cls}}, Confidence: {{box.conf}}, BBox: {{box.xyxy}}")
```

---

## 📦 Export

The model can be exported to multiple formats:

```python
from ultralytics import YOLO
model = YOLO("artifacts/best.pt")

model.export(format="onnx")        # ONNX format
model.export(format="torchscript") # TorchScript
model.export(format="tflite")      # TensorFlow Lite
model.export(format="engine")      # TensorRT
```

---

## ⚙️ Configuration

Edit `configs/default.yaml` to customize all parameters:

```yaml
paths:
  images_dir: data/images
  csv_path: data/annotations.csv
  output_dir: artifacts

training:
  model: yolov8n.pt
  imgsz: 640
  epochs: 50
  batch: 16
  seed: 42

data:
  split_ratio: 0.8
  allow_background: true
```

---

## 🔧 Troubleshooting

| Issue | Solution |
|-------|----------|
| `No matching images found` | Check that `images_dir` points to the correct folder and filenames in CSV match |
| `Cannot detect CSV format` | Ensure CSV has proper column headers (see Data Format section) |
| `CUDA out of memory` | Reduce `batch` size or use a smaller model (yolov8n) |
| `Invalid bounding boxes` | Check that coordinates are within image dimensions |
| `Low mAP scores` | Increase epochs, try larger model, or check annotation quality |

---

## 📄 License

This project is licensed under the {LICENSE_TYPE} License. See the [LICENSE](LICENSE) file for details.

---

## 🤝 Acknowledgments

- [Ultralytics YOLOv8](https://github.com/ultralytics/ultralytics)
- [YOLO Documentation](https://docs.ultralytics.com/)
"""

with open(os.path.join(REPO_DIR, "README.md"), 'w') as f:
    f.write(readme_content)
print("✅ README.md generated")

In [ ]:
# ── requirements.txt ─────────────────────────────────────────────
requirements = """ultralytics==8.2.103
opencv-python-headless==4.10.0.84
pandas==2.2.2
pyyaml==6.0.2
matplotlib==3.9.2
Pillow==10.4.0
tqdm==4.66.5
seaborn==0.13.2
onnx==1.16.2
onnxruntime==1.19.2
"""
with open(os.path.join(REPO_DIR, "requirements.txt"), 'w') as f:
    f.write(requirements.strip() + "\n")
print("✅ requirements.txt")

# ── .gitignore ───────────────────────────────────────────────────
gitignore = """# Python
__pycache__/
*.py[cod]
*.egg-info/
dist/
build/
*.egg
.eggs/

# Virtual environments
venv/
env/
.env

# IDE
.vscode/
.idea/
*.swp
*.swo

# OS
.DS_Store
Thumbs.db

# Data & Models (large files)
*.pt
*.onnx
*.torchscript
*.tflite
*.engine

# Dataset (user data)
data/images/
data/annotations.csv
dataset/

# Training runs
runs/
wandb/

# Jupyter
.ipynb_checkpoints/
"""
with open(os.path.join(REPO_DIR, ".gitignore"), 'w') as f:
    f.write(gitignore.strip() + "\n")
print("✅ .gitignore")

# ── LICENSE ──────────────────────────────────────────────────────
if LICENSE_TYPE == "MIT":
    license_text = """MIT License

Copyright (c) 2024

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""
else:
    license_text = """                                 Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
"""
with open(os.path.join(REPO_DIR, "LICENSE"), 'w') as f:
    f.write(license_text.strip() + "\n")
print("✅ LICENSE")

# ── pyproject.toml ───────────────────────────────────────────────
pyproject = """[build-system]
requires = ["setuptools>=68.0", "wheel"]
build-backend = "setuptools.build_meta"

[project]
name = "yolov8-pipeline"
version = "1.0.0"
description = "Professional YOLOv8 object detection training pipeline with CSV annotation support"
readme = "README.md"
requires-python = ">=3.8"
license = {text = """" + LICENSE_TYPE + """"}
authors = [{name = "YOLOv8 Pipeline Team"}]
keywords = ["yolov8", "object-detection", "computer-vision", "deep-learning"]
classifiers = [
    "Development Status :: 4 - Beta",
    "Intended Audience :: Developers",
    "Intended Audience :: Science/Research",
    "Programming Language :: Python :: 3",
    "Topic :: Scientific/Engineering :: Artificial Intelligence",
    "Topic :: Scientific/Engineering :: Image Recognition",
]
dependencies = [
    "ultralytics>=8.2",
    "opencv-python-headless>=4.8",
    "pandas>=2.0",
    "pyyaml>=6.0",
    "matplotlib>=3.7",
    "Pillow>=10.0",
    "tqdm>=4.65",
]

[project.optional-dependencies]
export = ["onnx>=1.14", "onnxruntime>=1.16"]
dev = ["pytest", "black", "ruff", "pre-commit"]

[tool.setuptools.packages.find]
where = ["src"]

[tool.black]
line-length = 100

[tool.ruff]
line-length = 100
target-version = "py38"
"""
with open(os.path.join(REPO_DIR, "pyproject.toml"), 'w') as f:
    f.write(pyproject.strip() + "\n")
print("✅ pyproject.toml")

In [ ]:
# ── Makefile ─────────────────────────────────────────────────────
makefile = """# YOLOv8 Pipeline Makefile
.PHONY: install convert train predict export clean help

help: ## Show this help message
\t@grep -E '^[a-zA-Z_-]+:.*?## .*$$' $(MAKEFILE_LIST) | sort | \\
\t\tawk 'BEGIN {FS = ":.*?## "}; {printf "\\033[36m%-20s\\033[0m %s\\n", $$1, $$2}'

install: ## Install dependencies
\tpip install -r requirements.txt

convert: ## Convert CSV annotations to YOLO format
\tpython scripts/convert_csv_to_yolo.py --config configs/default.yaml

train: ## Train YOLOv8 model
\tpython scripts/train.py --config configs/default.yaml

predict: ## Run inference on images
\tpython scripts/predict.py --weights artifacts/best.pt --source data/images

export: ## Export model to ONNX
\tpython -c "from ultralytics import YOLO; YOLO('artifacts/best.pt').export(format='onnx')"

clean: ## Clean generated files
\trm -rf runs/ dataset/ __pycache__/
\tfind . -type d -name __pycache__ -exec rm -rf {} + 2>/dev/null || true

lint: ## Run linter
\truff check src/ scripts/

format: ## Format code
\tblack src/ scripts/
"""
with open(os.path.join(REPO_DIR, "Makefile"), 'w') as f:
    f.write(makefile)
print("✅ Makefile")

In [ ]:
# ── src/yolov8_pipeline/__init__.py ──────────────────────────────
init_py = '''"""YOLOv8 Pipeline - Professional object detection training pipeline."""

__version__ = "1.0.0"

from .parser import detect_csv_format, normalize_columns, build_class_mapping
from .converter import convert_to_yolo, create_yolo_dataset
from .visualizer import visualize_yolo_labels

__all__ = [
    "detect_csv_format",
    "normalize_columns",
    "build_class_mapping",
    "convert_to_yolo",
    "create_yolo_dataset",
    "visualize_yolo_labels",
]
'''
with open(os.path.join(REPO_DIR, "src/yolov8_pipeline/__init__.py"), 'w') as f:
    f.write(init_py)

# ── src/yolov8_pipeline/parser.py ────────────────────────────────
parser_py = '''"""CSV annotation parsing and format detection."""

import os
import pandas as pd


def detect_csv_format(df: pd.DataFrame) -> str:
    """
    Auto-detect CSV annotation format.

    Returns:
        'A' - pixel boxes (xmin, ymin, xmax, ymax)
        'B' - pixel boxes (xmin, ymin, width, height)
        'C' - normalized YOLO format

    Raises:
        ValueError: If format cannot be determined.
    """
    cols_lower = [c.lower().strip() for c in df.columns]

    if "normalized" in cols_lower:
        return "C"
    if "xmax" in cols_lower and "ymax" in cols_lower:
        return "A"
    if "width" in cols_lower and "height" in cols_lower:
        return "B"

    if len(df.columns) >= 6:
        numeric_cols = df.iloc[:, 1:5]
        try:
            numeric_vals = numeric_cols.astype(float)
            if numeric_vals.max().max() <= 1.0 and numeric_vals.min().min() >= 0.0:
                return "C"
        except (ValueError, TypeError):
            pass
        return "A"

    raise ValueError(
        "Cannot detect CSV format. Expected columns:\n"
        "  Format A: filename, xmin, ymin, xmax, ymax, class\n"
        "  Format B: filename, xmin, ymin, width, height, class\n"
        "  Format C: filename, x_center, y_center, width, height, class, normalized"
    )


def normalize_columns(df: pd.DataFrame, fmt: str) -> pd.DataFrame:
    """Rename CSV columns to standard names based on detected format."""
    df = df.copy()
    cols = list(df.columns)
    cols_lower = {c.lower().strip(): c for c in cols}

    column_maps = {
        "A": {
            "filename": ["filename", "file", "image", "image_name", "img"],
            "xmin": ["xmin", "x_min", "x1", "left"],
            "ymin": ["ymin", "y_min", "y1", "top"],
            "xmax": ["xmax", "x_max", "x2", "right"],
            "ymax": ["ymax", "y_max", "y2", "bottom"],
            "class": ["class", "label", "class_name", "category", "cls"],
        },
        "B": {
            "filename": ["filename", "file", "image", "image_name", "img"],
            "xmin": ["xmin", "x_min", "x1", "left", "x"],
            "ymin": ["ymin", "y_min", "y1", "top", "y"],
            "width": ["width", "w", "box_width"],
            "height": ["height", "h", "box_height"],
            "class": ["class", "label", "class_name", "category", "cls"],
        },
        "C": {
            "filename": ["filename", "file", "image", "image_name", "img"],
            "x_center": ["x_center", "cx", "center_x", "x"],
            "y_center": ["y_center", "cy", "center_y", "y"],
            "width": ["width", "w", "box_width"],
            "height": ["height", "h", "box_height"],
            "class": ["class", "label", "class_name", "category", "cls"],
        },
    }

    mapping = {}
    for target, candidates in column_maps[fmt].items():
        for c in candidates:
            if c in cols_lower:
                mapping[cols_lower[c]] = target
                break

    if len(mapping) < 6:
        if fmt == "A":
            mapping = {cols[0]: "filename", cols[1]: "xmin", cols[2]: "ymin",
                       cols[3]: "xmax", cols[4]: "ymax", cols[5]: "class"}
        elif fmt == "B":
            mapping = {cols[0]: "filename", cols[1]: "xmin", cols[2]: "ymin",
                       cols[3]: "width", cols[4]: "height", cols[5]: "class"}
        elif fmt == "C":
            mapping = {cols[0]: "filename", cols[1]: "x_center", cols[2]: "y_center",
                       cols[3]: "width", cols[4]: "height", cols[5]: "class"}

    df = df.rename(columns=mapping)
    df["filename"] = df["filename"].apply(lambda x: os.path.basename(str(x).strip()))
    return df


def build_class_mapping(classes_series: pd.Series) -> tuple:
    """
    Build class label to integer ID mapping.

    Returns:
        Tuple of (mapping_dict, class_names_list)
    """
    unique_classes = classes_series.unique()

    all_int = True
    for c in unique_classes:
        try:
            int(c)
        except (ValueError, TypeError):
            all_int = False
            break

    if all_int:
        int_classes = sorted([int(c) for c in unique_classes])
        if int_classes == list(range(len(int_classes))):
            mapping = {str(c): c for c in int_classes}
            names = [str(c) for c in int_classes]
        else:
            mapping = {str(c): i for i, c in enumerate(int_classes)}
            names = [str(c) for c in int_classes]
    else:
        sorted_classes = sorted([str(c) for c in unique_classes])
        mapping = {c: i for i, c in enumerate(sorted_classes)}
        names = sorted_classes

    return mapping, names
'''
with open(os.path.join(REPO_DIR, "src/yolov8_pipeline/parser.py"), 'w') as f:
    f.write(parser_py)

print("✅ src/yolov8_pipeline/__init__.py")
print("✅ src/yolov8_pipeline/parser.py")

In [ ]:
# ── src/yolov8_pipeline/converter.py ─────────────────────────────
converter_py = '''"""YOLO format conversion utilities."""

import os
import random
import shutil
from pathlib import Path

import pandas as pd
import yaml
from PIL import Image
from tqdm.auto import tqdm

from .parser import detect_csv_format, normalize_columns, build_class_mapping


def convert_to_yolo(row: dict, fmt: str, img_w: int, img_h: int):
    """
    Convert a single annotation row to YOLO format.

    Returns:
        Tuple (x_center, y_center, width, height) normalized to [0, 1],
        or None if the box is invalid.
    """
    try:
        if fmt == "A":
            xmin, ymin = float(row["xmin"]), float(row["ymin"])
            xmax, ymax = float(row["xmax"]), float(row["ymax"])
        elif fmt == "B":
            xmin, ymin = float(row["xmin"]), float(row["ymin"])
            xmax = xmin + float(row["width"])
            ymax = ymin + float(row["height"])
        elif fmt == "C":
            xc = max(0.0, min(1.0, float(row["x_center"])))
            yc = max(0.0, min(1.0, float(row["y_center"])))
            w = max(0.0, min(1.0, float(row["width"])))
            h = max(0.0, min(1.0, float(row["height"])))
            return (xc, yc, w, h) if w > 0 and h > 0 else None
        else:
            return None

        xmin = max(0, min(img_w, xmin))
        ymin = max(0, min(img_h, ymin))
        xmax = max(0, min(img_w, xmax))
        ymax = max(0, min(img_h, ymax))

        bw, bh = xmax - xmin, ymax - ymin
        if bw <= 0 or bh <= 0:
            return None

        return (
            (xmin + xmax) / 2.0 / img_w,
            (ymin + ymax) / 2.0 / img_h,
            bw / img_w,
            bh / img_h,
        )
    except (ValueError, TypeError):
        return None


def create_yolo_dataset(
    images_dir: str,
    csv_path: str,
    output_dir: str,
    split_ratio: float = 0.8,
    seed: int = 42,
    allow_background: bool = True,
) -> str:
    """
    Create a complete YOLO dataset from images and CSV annotations.

    Returns:
        Path to the generated data.yaml file.
    """
    df = pd.read_csv(csv_path)
    fmt = detect_csv_format(df)
    df = normalize_columns(df, fmt)

    for col in df.columns:
        if col not in ("filename", "class"):
            df[col] = pd.to_numeric(df[col], errors="coerce")
    df = df.dropna()

    class_mapping, class_names = build_class_mapping(df["class"])

    image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"}
    available = {}
    for f in os.listdir(images_dir):
        if Path(f).suffix.lower() in image_extensions:
            available[f] = os.path.join(images_dir, f)

    df = df[df["filename"].isin(available)]

    for split in ("train", "val"):
        os.makedirs(os.path.join(output_dir, "images", split), exist_ok=True)
        os.makedirs(os.path.join(output_dir, "labels", split), exist_ok=True)

    all_images = sorted(df["filename"].unique())
    random.seed(seed)
    random.shuffle(all_images)
    split_idx = int(len(all_images) * split_ratio)
    train_set = set(all_images[:split_idx])
    val_set = set(all_images[split_idx:])

    if allow_background:
        bg = set(available.keys()) - set(df["filename"].unique())
        bg_list = sorted(bg)
        random.shuffle(bg_list)
        bg_split = int(len(bg_list) * split_ratio)
        train_set |= set(bg_list[:bg_split])
        val_set |= set(bg_list[bg_split:])

    for img_name in tqdm(sorted(train_set | val_set), desc="Converting"):
        split = "train" if img_name in train_set else "val"
        src = available.get(img_name)
        if not src:
            continue

        shutil.copy2(src, os.path.join(output_dir, "images", split, img_name))
        img_w, img_h = Image.open(src).size

        img_df = df[df["filename"] == img_name]
        lines = []
        for _, row in img_df.iterrows():
            result = convert_to_yolo(row, fmt, img_w, img_h)
            if result:
                cls_id = class_mapping[str(row["class"])]
                xc, yc, w, h = result
                lines.append(f"{cls_id} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}")

        label_name = Path(img_name).stem + ".txt"
        with open(os.path.join(output_dir, "labels", split, label_name), "w") as f:
            f.write("\n".join(lines))

    data_yaml = {
        "path": os.path.abspath(output_dir),
        "train": "images/train",
        "val": "images/val",
        "nc": len(class_names),
        "names": class_names,
    }
    yaml_path = os.path.join(output_dir, "data.yaml")
    with open(yaml_path, "w") as f:
        yaml.dump(data_yaml, f, default_flow_style=False, sort_keys=False)

    return yaml_path
'''
with open(os.path.join(REPO_DIR, "src/yolov8_pipeline/converter.py"), 'w') as f:
    f.write(converter_py)
print("✅ src/yolov8_pipeline/converter.py")

In [ ]:
# ── src/yolov8_pipeline/visualizer.py ────────────────────────────
visualizer_py = '''"""Visualization utilities for YOLO datasets."""

import os

import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image


def visualize_yolo_labels(
    image_path: str,
    label_path: str,
    class_names: list,
    ax=None,
    show: bool = False,
):
    """
    Draw YOLO bounding boxes on an image.

    Args:
        image_path: Path to the image file.
        label_path: Path to the YOLO label file.
        class_names: List of class names ordered by ID.
        ax: Matplotlib axes (created if None).
        show: Whether to call plt.show().

    Returns:
        Matplotlib axes with the visualization.
    """
    img = Image.open(image_path)
    w, h = img.size

    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(8, 8))

    ax.imshow(img)
    ax.set_title(os.path.basename(image_path), fontsize=9)
    ax.axis("off")

    colors = plt.cm.Set3(np.linspace(0, 1, max(len(class_names), 1)))

    if os.path.exists(label_path):
        with open(label_path) as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                cls_id = int(parts[0])
                xc, yc, bw, bh = [float(x) for x in parts[1:]]
                x1 = (xc - bw / 2) * w
                y1 = (yc - bh / 2) * h
                rect_w = bw * w
                rect_h = bh * h
                color = colors[cls_id % len(colors)]
                rect = patches.Rectangle(
                    (x1, y1), rect_w, rect_h,
                    linewidth=2, edgecolor=color, facecolor="none",
                )
                ax.add_patch(rect)
                label = class_names[cls_id] if cls_id < len(class_names) else str(cls_id)
                ax.text(
                    x1, y1 - 4, label, fontsize=8, color="white",
                    bbox=dict(boxstyle="round,pad=0.2", facecolor=color, alpha=0.8),
                )

    if show:
        plt.tight_layout()
        plt.show()

    return ax
'''
with open(os.path.join(REPO_DIR, "src/yolov8_pipeline/visualizer.py"), 'w') as f:
    f.write(visualizer_py)
print("✅ src/yolov8_pipeline/visualizer.py")

In [ ]:
# ── scripts/convert_csv_to_yolo.py ───────────────────────────────
convert_script = '''#!/usr/bin/env python3
"""Convert CSV annotations to YOLO format."""

import argparse
import sys
from pathlib import Path

import yaml

# Add src to path
sys.path.insert(0, str(Path(__file__).resolve().parent.parent / "src"))

from yolov8_pipeline.converter import create_yolo_dataset


def main():
    parser = argparse.ArgumentParser(description="Convert CSV annotations to YOLO format")
    parser.add_argument("--config", type=str, default="configs/default.yaml",
                        help="Path to config YAML file")
    parser.add_argument("--images-dir", type=str, help="Override images directory")
    parser.add_argument("--csv-path", type=str, help="Override CSV path")
    parser.add_argument("--output-dir", type=str, help="Override output directory")
    parser.add_argument("--split-ratio", type=float, help="Train/val split ratio")
    parser.add_argument("--seed", type=int, help="Random seed")
    args = parser.parse_args()

    # Load config
    with open(args.config) as f:
        config = yaml.safe_load(f)

    images_dir = args.images_dir or config["paths"]["images_dir"]
    csv_path = args.csv_path or config["paths"]["csv_path"]
    output_dir = args.output_dir or config["paths"].get("dataset_dir", "dataset")
    split_ratio = args.split_ratio or config["data"]["split_ratio"]
    seed = args.seed or config["data"].get("seed", 42)
    allow_bg = config["data"].get("allow_background", True)

    print(f"Converting CSV annotations to YOLO format...")
    print(f"  Images: {images_dir}")
    print(f"  CSV:    {csv_path}")
    print(f"  Output: {output_dir}")

    yaml_path = create_yolo_dataset(
        images_dir=images_dir,
        csv_path=csv_path,
        output_dir=output_dir,
        split_ratio=split_ratio,
        seed=seed,
        allow_background=allow_bg,
    )
    print(f"\nDone! data.yaml: {yaml_path}")


if __name__ == "__main__":
    main()
'''
with open(os.path.join(REPO_DIR, "scripts/convert_csv_to_yolo.py"), 'w') as f:
    f.write(convert_script)
print("✅ scripts/convert_csv_to_yolo.py")

# ── scripts/train.py ─────────────────────────────────────────────
train_script = '''#!/usr/bin/env python3
"""Train YOLOv8 model."""

import argparse
import os
import sys
from pathlib import Path

import yaml

sys.path.insert(0, str(Path(__file__).resolve().parent.parent / "src"))


def main():
    parser = argparse.ArgumentParser(description="Train YOLOv8 model")
    parser.add_argument("--config", type=str, default="configs/default.yaml",
                        help="Path to config YAML file")
    parser.add_argument("--data", type=str, help="Path to data.yaml")
    parser.add_argument("--model", type=str, help="Model size (e.g., yolov8n.pt)")
    parser.add_argument("--imgsz", type=int, help="Image size")
    parser.add_argument("--epochs", type=int, help="Number of epochs")
    parser.add_argument("--batch", type=int, help="Batch size")
    parser.add_argument("--device", type=str, help="Device (cuda/cpu)")
    args = parser.parse_args()

    with open(args.config) as f:
        config = yaml.safe_load(f)

    tc = config["training"]
    data_path = args.data or os.path.join(
        config["paths"].get("dataset_dir", "dataset"), "data.yaml"
    )
    model_name = args.model or tc["model"]
    imgsz = args.imgsz or tc["imgsz"]
    epochs = args.epochs or tc["epochs"]
    batch = args.batch or tc["batch"]
    device = args.device or tc.get("device", "")
    seed = tc.get("seed", 42)

    from ultralytics import YOLO

    print(f"Training YOLOv8...")
    print(f"  Model:  {model_name}")
    print(f"  Data:   {data_path}")
    print(f"  Epochs: {epochs}")
    print(f"  ImgSz:  {imgsz}")
    print(f"  Batch:  {batch}")

    model = YOLO(model_name)
    results = model.train(
        data=data_path,
        imgsz=imgsz,
        epochs=epochs,
        batch=batch,
        device=device if device else None,
        seed=seed,
        project="runs",
        name="train",
        exist_ok=True,
    )
    print("\nTraining complete!")
    print(f"Best weights: runs/train/weights/best.pt")


if __name__ == "__main__":
    main()
'''
with open(os.path.join(REPO_DIR, "scripts/train.py"), 'w') as f:
    f.write(train_script)
print("✅ scripts/train.py")

# ── scripts/predict.py ───────────────────────────────────────────
predict_script = '''#!/usr/bin/env python3
"""Run YOLOv8 inference on images."""

import argparse
import os


def main():
    parser = argparse.ArgumentParser(description="Run YOLOv8 inference")
    parser.add_argument("--weights", type=str, required=True,
                        help="Path to model weights (.pt)")
    parser.add_argument("--source", type=str, required=True,
                        help="Path to image, directory, or video")
    parser.add_argument("--conf", type=float, default=0.25,
                        help="Confidence threshold")
    parser.add_argument("--imgsz", type=int, default=640,
                        help="Inference image size")
    parser.add_argument("--save", action="store_true", default=True,
                        help="Save results")
    parser.add_argument("--save-dir", type=str, default="runs/predict",
                        help="Directory to save results")
    args = parser.parse_args()

    from ultralytics import YOLO

    print(f"Running inference...")
    print(f"  Weights: {args.weights}")
    print(f"  Source:  {args.source}")
    print(f"  Conf:    {args.conf}")

    model = YOLO(args.weights)
    results = model.predict(
        source=args.source,
        conf=args.conf,
        imgsz=args.imgsz,
        save=args.save,
        project=args.save_dir,
        name="results",
        exist_ok=True,
    )

    print(f"\nInference complete!")
    print(f"  Processed {len(results)} images")
    if args.save:
        print(f"  Results saved to: {args.save_dir}/results/")


if __name__ == "__main__":
    main()
'''
with open(os.path.join(REPO_DIR, "scripts/predict.py"), 'w') as f:
    f.write(predict_script)
print("✅ scripts/predict.py")

In [ ]:
# ── configs/default.yaml ─────────────────────────────────────────
default_config = f"""# YOLOv8 Pipeline Configuration
# ================================

paths:
  images_dir: data/images
  csv_path: data/annotations.csv
  dataset_dir: dataset
  output_dir: artifacts

training:
  model: {MODEL_SIZE}
  imgsz: {IMGSZ}
  epochs: {EPOCHS}
  batch: {BATCH}
  seed: {SEED}
  device: ""  # auto

data:
  split_ratio: {SPLIT_RATIO}
  allow_background: {str(ALLOW_BACKGROUND_IMAGES).lower()}

export:
  formats:
    - onnx
    - torchscript
"""
with open(os.path.join(REPO_DIR, "configs/default.yaml"), 'w') as f:
    f.write(default_config)
print("✅ configs/default.yaml")

In [ ]:
# ── docs/dataset_format.md ───────────────────────────────────────
dataset_format_doc = """# Dataset Format Guide

## Overview

This pipeline accepts image datasets with bounding box annotations provided in CSV format.
The CSV format is **automatically detected** based on column names.

---

## Image Requirements

- **Supported formats**: JPG, JPEG, PNG, BMP, TIFF, WebP
- **Location**: Place all images in a single directory
- **Naming**: Filenames in the CSV must match the image filenames (basename only)

---

## CSV Annotation Formats

### Format A: Pixel Coordinates (Corners)

| Column | Type | Description |
|--------|------|-------------|
| `filename` | string | Image filename (e.g., `img001.jpg`) |
| `xmin` | float | Left edge of bounding box (pixels) |
| `ymin` | float | Top edge of bounding box (pixels) |
| `xmax` | float | Right edge of bounding box (pixels) |
| `ymax` | float | Bottom edge of bounding box (pixels) |
| `class` | string/int | Class label or ID |

**Example:**
```csv
filename,xmin,ymin,xmax,ymax,class
img001.jpg,100,50,300,200,cat
img001.jpg,400,100,550,350,dog
img002.jpg,50,50,200,180,cat
```

### Format B: Pixel Coordinates (Top-Left + Size)

| Column | Type | Description |
|--------|------|-------------|
| `filename` | string | Image filename |
| `xmin` | float | Left edge (pixels) |
| `ymin` | float | Top edge (pixels) |
| `width` | float | Box width (pixels) |
| `height` | float | Box height (pixels) |
| `class` | string/int | Class label or ID |

**Example:**
```csv
filename,xmin,ymin,width,height,class
img001.jpg,100,50,200,150,cat
img001.jpg,400,100,150,250,dog
```

### Format C: Normalized YOLO Format

| Column | Type | Description |
|--------|------|-------------|
| `filename` | string | Image filename |
| `x_center` | float | Center X (normalized 0-1) |
| `y_center` | float | Center Y (normalized 0-1) |
| `width` | float | Box width (normalized 0-1) |
| `height` | float | Box height (normalized 0-1) |
| `class` | string/int | Class label or ID |
| `normalized` | any | Flag column (presence triggers Format C detection) |

**Example:**
```csv
filename,x_center,y_center,width,height,class,normalized
img001.jpg,0.3125,0.2604,0.3125,0.3125,cat,true
img001.jpg,0.7422,0.4688,0.2344,0.5208,dog,true
```

---

## Class Labels

- **String labels**: Automatically sorted alphabetically and mapped to integer IDs
- **Integer labels**: Validated for contiguous range starting from 0; remapped if necessary

---

## Multiple Objects Per Image

Multiple rows with the same filename represent multiple objects in that image.
Each row becomes one bounding box in the YOLO label file.

---

## Column Name Flexibility

The parser recognizes common column name variants:

| Standard | Also Accepted |
|----------|---------------|
| `filename` | `file`, `image`, `image_name`, `img` |
| `xmin` | `x_min`, `x1`, `left` |
| `ymin` | `y_min`, `y1`, `top` |
| `xmax` | `x_max`, `x2`, `right` |
| `ymax` | `y_max`, `y2`, `bottom` |
| `width` | `w`, `box_width` |
| `height` | `h`, `box_height` |
| `class` | `label`, `class_name`, `category`, `cls` |
"""
with open(os.path.join(REPO_DIR, "docs/dataset_format.md"), 'w') as f:
    f.write(dataset_format_doc)
print("✅ docs/dataset_format.md")

# ── docs/usage_guide.md ──────────────────────────────────────────
usage_guide_doc = """# Usage Guide

## Table of Contents

1. [Prerequisites](#prerequisites)
2. [Installation](#installation)
3. [Preparing Your Data](#preparing-your-data)
4. [Converting Annotations](#converting-annotations)
5. [Training](#training)
6. [Evaluation](#evaluation)
7. [Inference](#inference)
8. [Exporting Models](#exporting-models)
9. [Using as a Python Package](#using-as-a-python-package)

---

## Prerequisites

- Python 3.8 or higher
- CUDA-capable GPU (recommended for training)
- pip package manager

---

## Installation

```bash
# Clone the repository
git clone https://github.com/m0d9/yolov8-pipeline.git
cd yolov8-pipeline

# Install dependencies
pip install -r requirements.txt

# Or install as a package
pip install -e .
```

---

## Preparing Your Data

1. Create a `data/` directory in the project root
2. Place your images in `data/images/`
3. Create `data/annotations.csv` with your bounding box annotations

See `docs/dataset_format.md` for supported CSV formats.

---

## Converting Annotations

```bash
# Using the default config
python scripts/convert_csv_to_yolo.py --config configs/default.yaml

# With custom paths
python scripts/convert_csv_to_yolo.py \\
    --images-dir /path/to/images \\
    --csv-path /path/to/annotations.csv \\
    --output-dir /path/to/dataset \\
    --split-ratio 0.8
```

This creates the YOLO dataset structure with train/val splits and generates `data.yaml`.

---

## Training

```bash
# Using config file
python scripts/train.py --config configs/default.yaml

# With overrides
python scripts/train.py \\
    --data dataset/data.yaml \\
    --model yolov8s.pt \\
    --epochs 100 \\
    --batch 32 \\
    --imgsz 640
```

Training results are saved to `runs/train/`.

---

## Evaluation

After training, evaluate on the validation set:

```python
from ultralytics import YOLO

model = YOLO("runs/train/weights/best.pt")
metrics = model.val(data="dataset/data.yaml")

print(f"mAP50: {metrics.box.map50}")
print(f"mAP50-95: {metrics.box.map}")
```

---

## Inference

```bash
# On a single image
python scripts/predict.py --weights artifacts/best.pt --source image.jpg

# On a directory
python scripts/predict.py --weights artifacts/best.pt --source images/

# With custom confidence threshold
python scripts/predict.py --weights artifacts/best.pt --source images/ --conf 0.5
```

---

## Exporting Models

```python
from ultralytics import YOLO

model = YOLO("artifacts/best.pt")

# Export to various formats
model.export(format="onnx")         # ONNX
model.export(format="torchscript")  # TorchScript
model.export(format="tflite")       # TensorFlow Lite
model.export(format="engine")       # TensorRT (requires TensorRT)
```

---

## Using as a Python Package

```python
from yolov8_pipeline import (
    detect_csv_format,
    normalize_columns,
    build_class_mapping,
    convert_to_yolo,
    create_yolo_dataset,
    visualize_yolo_labels,
)

# Create a complete YOLO dataset from CSV
yaml_path = create_yolo_dataset(
    images_dir="data/images",
    csv_path="data/annotations.csv",
    output_dir="dataset",
    split_ratio=0.8,
    seed=42,
)

# Or use individual functions
import pandas as pd

df = pd.read_csv("data/annotations.csv")
fmt = detect_csv_format(df)
df = normalize_columns(df, fmt)
mapping, names = build_class_mapping(df["class"])
```
"""
with open(os.path.join(REPO_DIR, "docs/usage_guide.md"), 'w') as f:
    f.write(usage_guide_doc)
print("✅ docs/usage_guide.md")

In [ ]:
# ── Hugging Face Model Card ──────────────────────────────────────
model_card = f"""---
license: {LICENSE_TYPE.lower()}
tags:
  - object-detection
  - yolov8
  - ultralytics
  - computer-vision
  - pytorch
library_name: ultralytics
pipeline_tag: object-detection
---

# YOLOv8 Object Detection Model

## Model Description

This is a **YOLOv8** object detection model trained using the [Ultralytics](https://github.com/ultralytics/ultralytics) framework. The model was trained on a custom dataset with CSV-formatted annotations, converted to YOLO format using an automated pipeline.

### Model Details

- **Architecture**: YOLOv8 ({MODEL_SIZE.replace('.pt', '')})
- **Input Size**: {IMGSZ}x{IMGSZ}
- **Number of Classes**: {{num_classes}}
- **Classes**: {{', '.join(class_names)}}
- **Framework**: Ultralytics / PyTorch

## Training Details

| Parameter | Value |
|-----------|-------|
| Model | {MODEL_SIZE} |
| Image Size | {IMGSZ} |
| Epochs | {EPOCHS} |
| Batch Size | {BATCH} |
| Seed | {SEED} |
| Train/Val Split | {SPLIT_RATIO}/{1-SPLIT_RATIO:.1f} |

## Intended Use

This model is intended for object detection tasks on images similar to the training data. It can detect and localize objects with bounding boxes and class labels.

### Primary Use Cases

- Real-time object detection in images and video
- Batch processing of image datasets
- Integration into computer vision pipelines

### Limitations

- Performance may degrade on images significantly different from the training distribution
- The model is optimized for the specific classes it was trained on
- Real-time performance depends on hardware (GPU recommended)

## How to Use

### Python API

```python
from ultralytics import YOLO

# Load the model
model = YOLO("best.pt")

# Run inference
results = model.predict("image.jpg", conf=0.25)

# Process results
for r in results:
    boxes = r.boxes
    for box in boxes:
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        xyxy = box.xyxy[0].tolist()
        print(f"Class: {{cls}}, Confidence: {{conf:.2f}}, Box: {{xyxy}}")
```

### Command Line

```bash
yolo predict model=best.pt source=image.jpg conf=0.25
```

## Export Formats

The model can be exported to:

| Format | File | Notes |
|--------|------|-------|
| PyTorch | `best.pt` | Default format |
| ONNX | `best.onnx` | Cross-platform inference |
| TorchScript | `best.torchscript` | C++ deployment |
| TensorFlow Lite | `best.tflite` | Mobile deployment |
| TensorRT | `best.engine` | NVIDIA GPU optimization |

## Citation

If you use this model, please cite:

```bibtex
@software{{ultralytics_yolov8,
  author = {{Jocher, Glenn and Chaurasia, Ayush and Qiu, Jing}},
  title = {{Ultralytics YOLOv8}},
  year = {{2023}},
  url = {{https://github.com/ultralytics/ultralytics}},
  license = {{AGPL-3.0}}
}}
```

## License

This model is released under the {LICENSE_TYPE} license. See the LICENSE file for details.

> **Note**: The Ultralytics YOLOv8 framework is licensed under AGPL-3.0.
> If you use the pretrained weights, please review the Ultralytics license terms.
"""

# Save as model-card.md in repo root
with open(os.path.join(REPO_DIR, "model-card.md"), 'w') as f:
    f.write(model_card)

# Also save as README.md in huggingface/ folder
with open(os.path.join(REPO_DIR, "huggingface/README.md"), 'w') as f:
    f.write(model_card)

print("✅ model-card.md")
print("✅ huggingface/README.md")

In [ ]:
# ── Copy artifacts into repository ───────────────────────────────
artifacts_repo = os.path.join(REPO_DIR, "artifacts")

for fname in os.listdir(OUTPUT_DIR):
    src = os.path.join(OUTPUT_DIR, fname)
    dst = os.path.join(artifacts_repo, fname)
    shutil.copy2(src, dst)
    print(f"   📦 {fname}")

# Copy data.yaml
shutil.copy2(data_yaml_path, os.path.join(artifacts_repo, "data.yaml"))

print(f"\n✅ Artifacts copied to {artifacts_repo}")

In [ ]:
# ── Display final repository structure ───────────────────────────
def print_tree(path, prefix="", max_depth=4, depth=0):
    if depth >= max_depth:
        return
    entries = sorted(os.listdir(path))
    dirs = [e for e in entries if os.path.isdir(os.path.join(path, e))]
    files = [e for e in entries if os.path.isfile(os.path.join(path, e))]

    for f in files:
        size = os.path.getsize(os.path.join(path, f))
        size_str = f"{size/1024:.1f}KB" if size > 1024 else f"{size}B"
        print(f"{prefix}📄 {f} ({size_str})")

    for i, d in enumerate(dirs):
        is_last = (i == len(dirs) - 1)
        print(f"{prefix}📂 {d}/")
        new_prefix = prefix + ("    " if is_last else "│   ")
        print_tree(os.path.join(path, d), new_prefix, max_depth, depth + 1)

print(f"\n📁 Repository Structure: {REPO_DIR}")
print("=" * 60)
print_tree(REPO_DIR)

## 11 · Package & Download

Zip the repository and artifacts for download.

In [ ]:
# ── Zip the repository ───────────────────────────────────────────
import zipfile

def zip_directory(source_dir, output_path):
    """Create a zip file from a directory."""
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(source_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, os.path.dirname(source_dir))
                zipf.write(file_path, arcname)
    size_mb = os.path.getsize(output_path) / (1024 * 1024)
    print(f"   📦 {output_path} ({size_mb:.2f} MB)")

print("Packaging repository and artifacts...\n")

# Zip repository
repo_zip = "/content/yolov8_pipeline_repo.zip"
zip_directory(REPO_DIR, repo_zip)

# Zip artifacts only
artifacts_zip = "/content/yolov8_trained_artifacts.zip"
zip_directory(OUTPUT_DIR, artifacts_zip)

print(f"\n✅ Ready for download!")
print(f"   📥 Repository:  {repo_zip}")
print(f"   📥 Artifacts:   {artifacts_zip}")

# Download in Colab
try:
    from google.colab import files
    print("\n⬇️  Downloading repository zip...")
    files.download(repo_zip)
    print("⬇️  Downloading artifacts zip...")
    files.download(artifacts_zip)
except ImportError:
    print("\n💡 Not running in Colab. Files are saved at the paths above.")

## 12 · Summary

### What was created:

| Component | Description |
|-----------|-------------|
| **YOLO Dataset** | Images + labels in train/val splits with `data.yaml` |
| **Trained Model** | `best.pt` weights with logged metrics |
| **Exports** | ONNX and TorchScript formats |
| **Repository** | Complete project structure for GitHub & Hugging Face |
| **Documentation** | README, dataset format guide, usage guide, model card |
| **Scripts** | Standalone conversion, training, and prediction scripts |
| **Package** | Installable Python package with reusable functions |
| **Configs** | YAML configuration for reproducible experiments |

### Next Steps:

1. **GitHub**: Unzip the repository, `git init`, and push to GitHub
2. **Hugging Face**: Upload the `huggingface/` folder contents to a new model repository
3. **Fine-tune**: Adjust hyperparameters in `configs/default.yaml` and retrain
4. **Deploy**: Use the ONNX export for production inference

---
> 🎉 **Pipeline complete!** Your professional YOLOv8 repository is ready.